In [3]:
import pandas as pd
covid_data = pd.read_csv(r'C:\Users\User\Desktop\p2\git\my_works\data\база\covid_data.csv')
covid_data.head()
vaccinations_data = pd.read_csv(r'C:\Users\User\Desktop\p2\git\my_works\data\база\country_vaccinations.csv')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]
covid_data = covid_data.groupby(['date','country'], as_index=False)[['confirmed', 'deaths', 'recovered']].sum()
covid_data['date']= pd.to_datetime(covid_data['date'])
covid_data['active']= covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']#еще болеют
covid_data = covid_data.sort_values(by= ['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()#прирост заболеваемости
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()#прирост смертности
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()#прирост выздоровевших
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])
covid_df = covid_data.merge(vaccinations_data, how='left', on=['date', 'country'])
covid_df['death_rate'] = covid_df['deaths'] /covid_df['confirmed']*100
covid_df['recover_rate'] = covid_df['recovered']/covid_df['confirmed']*100
#print(round(covid_df[covid_df['country'] == 'United States']['death_rate'].max(), 2))
#print(round(covid_df[covid_df['country']=='Russia']['recover_rate'].mean(), 2))
grouped_cases = covid_df.groupby('date')['daily_confirmed'].sum()
import matplotlib.pyplot as plt
import seaborn as sns
countries = ['Russia', 'Australia', 'Germany', 'Canada', 'United Kingdom']
croped_covid_df = covid_df[covid_df['country'].isin(countries)]

populations = pd.DataFrame([
    ['Canada', 37664517],
    ['Germany', 83721496],
    ['Russia', 145975300],
    ['Australia', 25726900],
    ['United Kingdom', 67802690]
    ],
    columns=['country', 'population']
)
croped_covid_df = croped_covid_df.merge(populations, on=['country'])
croped_covid_df['daily_confirmed_per_hundred'] = croped_covid_df['daily_confirmed'] / croped_covid_df['population'] * 100
import plotly
import plotly.express as px
line_data = covid_df.groupby('date', as_index=False).sum()
fig = px.line(
    data_frame=line_data, 
    x='date', 
    y=['confirmed', 'recovered', 'deaths', 'active'], 
    height=500, 
    width=1000, 
    title='Confirmed, Recovered, Deaths, Active cases over Time')
fig.show()


C:\Temp\ipykernel_8520\971205337.py:43: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed